In [3]:
import sys

!{sys.executable} -m pip install httpx
!{sys.executable} -m pip install --pre --index-url https://www.myget.org/F/apollo3zehn-dev/python/ nexusapi

You should consider upgrading via the 'C:\Users\wilvin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://www.myget.org/F/apollo3zehn-dev/python/


You should consider upgrading via the 'C:\Users\wilvin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
from nexusapi import NexusAsyncClient

base_url = "https://localhost:8443"
client = NexusAsyncClient.create(base_url):